In [1]:
pip install pandas_ta

     |████████████████████████████████| 92kB 6.2MB/s 
  Created wheel for pandas-ta: filename=pandas_ta-0.2.45b0-cp37-none-any.whl size=175305 sha256=82e42824ed81db9236f6f9348d66e7312b842a27eecd06571c88ef46102dab38
  Stored in directory: /root/.cache/pip/wheels/78/76/74/e4b09c98fc91d7e1e0a56f4fe067796f216deacaa509234579
Successfully built pandas-ta


In [2]:
pip install yfinance

     |████████████████████████████████| 6.3MB 10.1MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=eaafcd662896119bf0bceeeefa4d3e480ba0909e42383773220495f8a4093783
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [3]:
import pandas as pd
# import yfinance as yf
import datetime as dt
import pandas_ta as ta


# This function finds the buy sell
# schedule for maximum profit
def stockBuySell(fdf):
    
    # start = dt.datetime.today()-dt.timedelta(7)
    # end = dt.datetime.today()
    # fdf = yf.download(tick,start=start, end=end,interval='1m')
    # fdf.ta.strategy(ta.AllStrategy)
    price = fdf['close']
    n = len(fdf)
    
    fdf['final_buy'] = 'no'
    # fdf['final_sell'] = 'no'
    # fdf['profit'] = 0
    total = 0
    # Prices must be given for at least two days
    if (n == 1):
        return
    
    # Traverse through given price array
    i = 0
    while (i < (n - 1)):
        
        # Find Local Minima
        # Note that the limit is (n-2) as we are
        # comparing present element to the next element
        while ((i < (n - 1)) and
                (price[i + 1] <= price[i])):
            i += 1
        
        # If we reached the end, break
        # as no further solution possible
        if (i == n - 1):
            break
        
        # Store the index of minima
        buy = i
        i += 1
        
        # Find Local Maxima
        # Note that the limit is (n-1) as we are
        # comparing to previous element
        while ((i < n) and (price[i] >= price[i - 1])):
            i += 1
            
        # Store the index of maxima
        sell = i - 1
        
        profit = fdf['close'][sell] - fdf['close'][buy]
        
        if (profit >= 5):
            
            print("Buy on day: ",buy,"\t", "Sell on day: ",sell,"\t", "Profit: ",profit)  
            total = total + profit
            # print(total)
            fdf['final_buy'][buy] = 'buy'
            fdf['final_buy'][sell] = 'sell'
            # fdf['profit'][sell] = profit
    # fdf.drop(['open', 'high', 'low', 'close', 'adj_close', 'volume','HILOs_13_21','HILOl_13_21','SUPERTl_7_3.0','QQEs_14_5_4.236','PSARs_0.02_0.2','PSARl_0.02_0.2','SUPERTs_7_3.0','QQEl_14_5_4.236'],axis=1,inplace=True)
    # fdf = fdf.dropna()
    # fdf = fdf.reset_index(drop=True)
    return fdf   

In [4]:
link = 'https://raw.githubusercontent.com/rushiai/data_csv/main/AXISBANK__EQ__NSE__NSE__MINUTE.csv'
df = pd.read_csv(link)
df = df.set_index(['timestamp'])
df.index = pd.to_datetime(df.index)
# df.head()
vol = df.resample('15min').sum().volume
price = df['close'].resample('15Min').ohlc()
ndf = pd.concat([price, vol], axis=1)
ndf.dropna(inplace=True)

In [5]:
# temp = ndf.iloc[17698:]
# train = temp.iloc[0:6000]
# test = temp.iloc[6001:]

temp = ndf
train = temp
# test = temp.iloc[6001:]

In [ ]:
train.ta.strategy(ta.AllStrategy)
# test.ta.strategy(ta.AllStrategy)

In [ ]:
trainp = stockBuySell(train)
# testp = stockBuySell(test)

In [ ]:
trainp.drop(['QQEs_14_5_4.236', 'PSARs_0.02_0.2', 'SUPERTl_7_3.0', 'SUPERTs_7_3.0',
       'PSARl_0.02_0.2', 'QQEl_14_5_4.236', 'HILOl_13_21', 'HILOs_13_21','open','volume'],inplace=True,axis=1)
# testp.drop(['QQEs_14_5_4.236', 'PSARs_0.02_0.2', 'SUPERTl_7_3.0', 'SUPERTs_7_3.0',
    #    'PSARl_0.02_0.2', 'QQEl_14_5_4.236', 'HILOl_13_21', 'HILOs_13_21','open','volume'],inplace=True,axis=1)
trainp.dropna(inplace=True)
# testp.dropna(inplace=True)

In [ ]:
X = trainp.drop(['final_buy'],axis=1)
y = trainp.final_buy

In [ ]:
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import NearMiss

In [ ]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_res,y_res=smk.fit_sample(X,y)

In [ ]:
X_res.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_trainXG, X_testXG, y_trainXG, y_testXG = train_test_split(X_res,y_res, test_size = 0.2, random_state = 0)

In [ ]:
import xgboost as xgb
my_model = xgb.XGBClassifier()
my_model.fit(X_trainXG, y_trainXG)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [ ]:
# Predicting the Test set results
y_pred = my_model.predict(X_testXG)
from sklearn.metrics import accuracy_score
accuracy_score(y_testXG, y_pred)

0.9630161851459689

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_testXG, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.97      0.98      0.97      4394
     class 1       0.96      0.92      0.94      4336
     class 2       0.96      0.99      0.97      4492

    accuracy                           0.96     13222
   macro avg       0.96      0.96      0.96     13222
weighted avg       0.96      0.96      0.96     13222



In [ ]:
import yfinance as yf

In [ ]:
start = dt.datetime.today()-dt.timedelta(59)
end = dt.datetime.today()
fdf = yf.download('AXISBANK.NS',start=start, end=end,interval='15m')
fdf.ta.strategy(ta.AllStrategy)

[*********************100%***********************]  1 of 1 completed


/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:1092: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,
/usr/local/lib/python3.7/dist-packages/pandas/core/arrays/datetimes.py:1092: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  UserWarning,


In [ ]:
axis = stockBuySell(fdf)

Buy on day:  20 	 Sell on day:  23 	 Profit:  8.0999755859375
Buy on day:  24 	 Sell on day:  27 	 Profit:  16.60003662109375
Buy on day:  34 	 Sell on day:  39 	 Profit:  8.45001220703125
Buy on day:  40 	 Sell on day:  42 	 Profit:  5.9000244140625
Buy on day:  58 	 Sell on day:  60 	 Profit:  6.39996337890625
Buy on day:  61 	 Sell on day:  65 	 Profit:  7.95001220703125
Buy on day:  75 	 Sell on day:  78 	 Profit:  13.25
Buy on day:  81 	 Sell on day:  88 	 Profit:  5.20001220703125
Buy on day:  97 	 Sell on day:  103 	 Profit:  25.45001220703125
Buy on day:  105 	 Sell on day:  107 	 Profit:  5.75
Buy on day:  117 	 Sell on day:  120 	 Profit:  6.9000244140625
Buy on day:  123 	 Sell on day:  125 	 Profit:  6.75
Buy on day:  131 	 Sell on day:  134 	 Profit:  10.64996337890625
Buy on day:  147 	 Sell on day:  148 	 Profit:  5.54998779296875
Buy on day:  149 	 Sell on day:  151 	 Profit:  7.39996337890625
Buy on day:  161 	 Sell on day:  167 	 Profit:  9.64996337890625
Buy on day: 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:62: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
axis.drop(['QQEs_14_5_4.236', 'PSARs_0.02_0.2', 'SUPERTl_7_3.0', 'SUPERTs_7_3.0',
       'PSARl_0.02_0.2', 'QQEl_14_5_4.236', 'HILOl_13_21', 'HILOs_13_21','open','volume'],inplace=True,axis=1)
axis.dropna(inplace=True)

In [ ]:
ax = axis.drop(['final_buy'],axis=1)
ay = axis.final_buy

In [ ]:
ax.drop(['VTXM_14'],axis=1,inplace=True)

In [ ]:
ax.shape

(847, 186)

In [ ]:
X_res.shape

(66109, 186)

In [ ]:
res_col = ['f0', 'f1', 'f2', 'f3', 'f4', 'f5', 'f6', 'f7', 'f8', 'f9', 'f10', 'f11', 'f12', 'f13', 'f14', 'f15', 'f16', 'f17', 'f18', 'f19', 'f20', 'f21', 'f22', 'f23', 'f24', 'f25', 'f26', 'f27', 'f28', 'f29', 'f30', 'f31', 'f32', 'f33', 'f34', 'f35', 'f36', 'f37', 'f38', 'f39', 'f40', 'f41', 'f42', 'f43', 'f44', 'f45', 'f46', 'f47', 'f48', 'f49', 'f50', 'f51', 'f52', 'f53', 'f54', 'f55', 'f56', 'f57', 'f58', 'f59', 'f60', 'f61', 'f62', 'f63', 'f64', 'f65', 'f66', 'f67', 'f68', 'f69', 'f70', 'f71', 'f72', 'f73', 'f74', 'f75', 'f76', 'f77', 'f78', 'f79', 'f80', 'f81', 'f82', 'f83', 'f84', 'f85', 'f86', 'f87', 'f88', 'f89', 'f90', 'f91', 'f92', 'f93', 'f94', 'f95', 'f96', 'f97', 'f98', 'f99', 'f100', 'f101', 'f102', 'f103', 'f104', 'f105', 'f106', 'f107', 'f108', 'f109', 'f110', 'f111', 'f112', 'f113', 'f114', 'f115', 'f116', 'f117', 'f118', 'f119', 'f120', 'f121', 'f122', 'f123', 'f124', 'f125', 'f126', 'f127', 'f128', 'f129', 'f130', 'f131', 'f132', 'f133', 'f134', 'f135', 'f136', 'f137', 'f138', 'f139', 'f140', 'f141', 'f142', 'f143', 'f144', 'f145', 'f146', 'f147', 'f148', 'f149', 'f150', 'f151', 'f152', 'f153', 'f154', 'f155', 'f156', 'f157', 'f158', 'f159', 'f160', 'f161', 'f162', 'f163', 'f164', 'f165', 'f166', 'f167', 'f168', 'f169', 'f170', 'f171', 'f172', 'f173', 'f174', 'f175', 'f176', 'f177', 'f178', 'f179', 'f180', 'f181', 'f182', 'f183', 'f184', 'f185']

In [ ]:
ax_col =  ax.columns

In [ ]:
ccc = zip(ax_col, res_col)
a_dictionary = dict(ccc)

In [ ]:
ax = ax[X.columns]

In [ ]:
ax.rename(columns = a_dictionary, inplace = True)

In [ ]:
# Implementing Oversampling for Handling Imbalanced 
smk = SMOTETomek(random_state=42)
X_org,y_org=smk.fit_sample(ax,ay)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


In [ ]:
# Predicting the Test set results
y_pred = my_model.predict(X_org)
from sklearn.metrics import accuracy_score
accuracy_score(y_org, y_pred)

0.3687374749498998

In [ ]:
from sklearn.metrics import classification_report
target_names = ['class 0', 'class 1', 'class 2']
print(classification_report(y_org, y_pred, target_names=target_names))

              precision    recall  f1-score   support

     class 0       0.50      0.10      0.16       672
     class 1       0.36      0.97      0.53       646
     class 2       0.34      0.06      0.10       678

    accuracy                           0.37      1996
   macro avg       0.40      0.38      0.27      1996
weighted avg       0.40      0.37      0.26      1996



In [ ]:
my_model.predict(ax.iloc[0:])

array(['no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'sell', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'sell', 'no', 'no', 'sell', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'sell', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no', 'no',
       'no', 'no', 'no', 'no', 'no', 'no', 'no', 

In [ ]:
ay[ay == 'buy']

datetime
2021-03-05 10:45:00+05:30    buy
2021-03-05 14:45:00+05:30    buy
2021-03-08 10:30:00+05:30    buy
2021-03-08 13:30:00+05:30    buy
2021-03-08 15:00:00+05:30    buy
                            ... 
2021-04-26 12:30:00+05:30    buy
2021-04-26 14:15:00+05:30    buy
2021-04-27 10:00:00+05:30    buy
2021-04-28 11:15:00+05:30    buy
2021-04-28 13:30:00+05:30    buy
Name: final_buy, Length: 67, dtype: object

In [ ]:
ay.shape

(847,)

In [ ]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical
from matplotlib import pyplot

# load a single file as a numpy array
def load_file(filepath):
	dataframe = read_csv(filepath, header=None, delim_whitespace=True)
	return dataframe.values

# load a list of files and return as a 3d numpy array
def load_group(filenames, prefix=''):
	loaded = list()
	for name in filenames:
		data = load_file(prefix + name)
		loaded.append(data)
	# stack group so that features are the 3rd dimension
	loaded = dstack(loaded)
	return loaded

# load a dataset group, such as train or test
def load_dataset_group(group, prefix=''):
	filepath = prefix + group + '/Inertial Signals/'
	# load all 9 files as a single array
	filenames = list()
	# total acceleration
	filenames += ['total_acc_x_'+group+'.txt', 'total_acc_y_'+group+'.txt', 'total_acc_z_'+group+'.txt']
	# body acceleration
	filenames += ['body_acc_x_'+group+'.txt', 'body_acc_y_'+group+'.txt', 'body_acc_z_'+group+'.txt']
	# body gyroscope
	filenames += ['body_gyro_x_'+group+'.txt', 'body_gyro_y_'+group+'.txt', 'body_gyro_z_'+group+'.txt']
	# load input data
	X = load_group(filenames, filepath)
	# load class output
	y = load_file(prefix + group + '/y_'+group+'.txt')
	return X, y

# load the dataset, returns train and test X and y elements
def load_dataset(prefix=''):
	# load all train
	trainX, trainy = load_dataset_group('train', prefix + 'HARDataset/')
	print(trainX.shape, trainy.shape)
	# load all test
	testX, testy = load_dataset_group('test', prefix + 'HARDataset/')
	print(testX.shape, testy.shape)
	# zero-offset class values
	trainy = trainy - 1
	testy = testy - 1
	# one hot encode y
	trainy = to_categorical(trainy)
	testy = to_categorical(testy)
	print(trainX.shape, trainy.shape, testX.shape, testy.shape)
	return trainX, trainy, testX, testy

# fit and evaluate a model
def evaluate_model(trainX, trainy, testX, testy):
	verbose, epochs, batch_size = 0, 15, 64
	n_timesteps, n_features, n_outputs = trainX.shape[1], trainX.shape[2], trainy.shape[1]
	model = Sequential()
	model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
	model.add(Dropout(0.5))
	model.add(Dense(100, activation='relu'))
	model.add(Dense(n_outputs, activation='softmax'))
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	# fit network
	model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose)
	# evaluate model
	_, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
	return accuracy

# summarize scores
def summarize_results(scores):
	print(scores)
	m, s = mean(scores), std(scores)
	print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

# run an experiment
def run_experiment(repeats=10):
	# load data
	trainX, trainy, testX, testy = load_dataset()
	# repeat experiment
	scores = list()
	for r in range(repeats):
		score = evaluate_model(trainX, trainy, testX, testy)
		score = score * 100.0
		print('>#%d: %.3f' % (r+1, score))
		scores.append(score)
	# summarize results
	summarize_results(scores)

# run the experiment
run_experiment()